## 1. Setup

In [1]:
import sys
sys.path.append('../..')

In [2]:
import config
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import warnings

from neural_networks.unet import UNet
from neural_networks.net_utils import predict_density_maps_and_get_counts
from utils.data.data_generator import DataGenerator
from utils.evaluation.evaluation import evaluation_results_as_dict
from utils.evaluation.evaluation import evaluation_results_as_df
from utils.input_output.io import load_images_and_density_maps
from utils.input_output.io import read_json, write_json
from utils.input_output.io import load_gt_counts

Using TensorFlow backend.


In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

warnings.filterwarnings('ignore')

## 2. Load the dataset

In [4]:
params = {
    'dim': config.IMG_DIM,
    'batch_size': 1,
    'patches_per_image': 1,
    'density_map_multiplication_factor': config.DENSITY_MAP_MULTIPLICATION_FACTOR,
    'shuffle': False,
    'data_augmentation': False
}

In [5]:
train_generator = DataGenerator(config.DATASET_PATH, 'train', **params)
val_generator = DataGenerator(config.DATASET_PATH, 'val', **params)
test_generator = DataGenerator(config.DATASET_PATH, 'test', **params)

In [6]:
train_gt_counts = load_gt_counts(config.TRAIN_GT_COUNT_PATH)
val_gt_counts = load_gt_counts(config.VAL_GT_COUNT_PATH)
test_gt_counts = load_gt_counts(config.TEST_GT_COUNT_PATH)

### Select the checkpoint file that you want to test/evaluate

In [7]:
checkpoint_filenames = sorted(os.listdir(config.CHECKPOINTS_PATH))
print(checkpoint_filenames)

['model.01-0.194.hdf5', 'model.02-0.174.hdf5', 'model.03-0.150.hdf5', 'model.04-0.151.hdf5', 'model.05-0.144.hdf5', 'model.06-0.134.hdf5', 'model.07-0.123.hdf5', 'model.08-0.127.hdf5', 'model.09-0.156.hdf5', 'model.10-0.129.hdf5', 'model.11-0.118.hdf5', 'model.12-0.127.hdf5', 'model.13-0.112.hdf5', 'model.14-0.110.hdf5', 'model.15-0.103.hdf5', 'model.16-0.104.hdf5', 'model.17-0.108.hdf5', 'model.18-0.103.hdf5', 'model.19-0.115.hdf5', 'model.20-0.102.hdf5', 'model.21-0.099.hdf5', 'model.22-0.101.hdf5', 'model.23-0.097.hdf5', 'model.24-0.094.hdf5', 'model.25-0.101.hdf5', 'model.26-0.094.hdf5', 'model.27-0.093.hdf5', 'model.28-0.111.hdf5', 'model.29-0.089.hdf5', 'model.30-0.090.hdf5', 'model.31-0.092.hdf5', 'model.32-0.089.hdf5', 'model.33-0.098.hdf5', 'model.34-0.095.hdf5', 'model.35-0.086.hdf5', 'model.36-0.099.hdf5', 'model.37-0.086.hdf5', 'model.38-0.093.hdf5', 'model.39-0.085.hdf5', 'model.40-0.088.hdf5', 'model.41-0.085.hdf5', 'model.42-0.084.hdf5', 'model.43-0.089.hdf5', 'model.44-

In [8]:
for checkpoint_idx in range(30, 50, 1):
    selected_checkpoint_filename = checkpoint_filenames[checkpoint_idx]
    print(f'selected checkpoint_filename: {selected_checkpoint_filename}')
    
    epoch = selected_checkpoint_filename.split('.')[1].split('-')[0]
    print('epoch:', epoch)
    
    # Set epoch and val loss
    CHECKPOINT_FILENAME = f'{config.CHECKPOINTS_PATH}/{selected_checkpoint_filename}'
    QUANTITATIVE_RESULTS_PATH = f'./{config.SUB_EXPERIMENT_NAME}/results/quantitative/epoch_{epoch}'
    
    !rm -rf $QUANTITATIVE_RESULTS_PATH
    os.makedirs(QUANTITATIVE_RESULTS_PATH)
    
    ## 3. Load the best model
    model = UNet(pretrained_weights=CHECKPOINT_FILENAME)

    train_pred_counts = predict_density_maps_and_get_counts(model, train_generator,
                                                        config.DENSITY_MAP_MULTIPLICATION_FACTOR)
    val_pred_counts = predict_density_maps_and_get_counts(model, val_generator,
                                                          config.DENSITY_MAP_MULTIPLICATION_FACTOR)
    test_pred_counts = predict_density_maps_and_get_counts(model, test_generator,
                                                           config.DENSITY_MAP_MULTIPLICATION_FACTOR)
    
    ## 4. Predict and evaluate
    train_results = evaluation_results_as_dict(train_gt_counts, train_pred_counts, 'train')
    val_results = evaluation_results_as_dict(val_gt_counts, val_pred_counts, 'val')
    test_results = evaluation_results_as_dict(test_gt_counts, test_pred_counts, 'test')

    df = evaluation_results_as_df(train_results, val_results, test_results,
                                  config.ARCHITECTURE_NAME,
                                  config.SUB_EXPERIMENT_NAME,
                                  config.DATASET_NAME)

    df.to_csv(f'{QUANTITATIVE_RESULTS_PATH}/results.csv', index=True)

selected checkpoint_filename: model.31-0.092.hdf5
epoch: 31
selected checkpoint_filename: model.32-0.089.hdf5
epoch: 32
selected checkpoint_filename: model.33-0.098.hdf5
epoch: 33
selected checkpoint_filename: model.34-0.095.hdf5
epoch: 34
selected checkpoint_filename: model.35-0.086.hdf5
epoch: 35
selected checkpoint_filename: model.36-0.099.hdf5
epoch: 36
selected checkpoint_filename: model.37-0.086.hdf5
epoch: 37
selected checkpoint_filename: model.38-0.093.hdf5
epoch: 38
selected checkpoint_filename: model.39-0.085.hdf5
epoch: 39
selected checkpoint_filename: model.40-0.088.hdf5
epoch: 40
selected checkpoint_filename: model.41-0.085.hdf5
epoch: 41
selected checkpoint_filename: model.42-0.084.hdf5
epoch: 42
selected checkpoint_filename: model.43-0.089.hdf5
epoch: 43
selected checkpoint_filename: model.44-0.085.hdf5
epoch: 44
selected checkpoint_filename: model.45-0.085.hdf5
epoch: 45
selected checkpoint_filename: model.46-0.086.hdf5
epoch: 46
selected checkpoint_filename: model.47-0